# Part B: Shortest Path

### Procedure
1. Select a city and save its street network in a Python graph G.
2. Plot the street network
3. Define a function named RoutePlan that receives the following parameters:
- a) Graph G.
- b) Depot location D
- c) A list L with the delivery locations.

4. RoutePlan returns the route that will complete the deliveries in the shortest possible time.
5. Plot a map of the street network of the selected city with the route highlighted.

# 1. Select a city and save its street network in a Python graph G.

In [ ]:
# importing the relevant libraries

import pandas as pd
import geopandas as gpd
import osmnx as ox
import networkx as nx
import folium

In [ ]:
# obtaining the driving street networking within 3000m of the Richmond, a suburb in East Melbourne
G = ox.graph_from_address('Richmond, Melbourne', network_type='drive', dist = 3000)

# 2. Plot the street network

In [ ]:
# plotting the street network
ox.plot_graph(G, node_size=5)

In [ ]:
# plotting the street network on an interactive map using folium
st_net_m = ox.folium.plot_graph_folium(G, graph_map=None, popup_attribute='name')
st_net_m

# 3. Define a function named RoutePlan that receives the following parameters:
## a) Graph G,  b) Depot location D,  c) A list L with the delivery locations.

### Data manipulation and cleaning
Exploring the street network data

In [ ]:
# copying the graph 'G' and storing it in the variable 'G_Rich'
G_Rich = G.copy()

In [ ]:
# creating geodataframes for nodes and edges in the network
nodes = ox.graph_to_gdfs(G_Rich, nodes=True, edges=False)
edges = ox.graph_to_gdfs(G_Rich, nodes=False, edges=True)

In [ ]:
# displaying the number of null values for each column of the geodataframes
nodes.isna().sum()
edges.isna().sum()

In [ ]:
print("Out of " + str(len(edges)) + " edges, there are " + str(edges["maxspeed"].isnull().sum()) + " edges that don't have a maxspeed assigned") 

Let assign maxspeeds to edges with missing data

### Impute missing edge speeds and calculate edge travel times

The following code and principles are adopted from G Boeing:

- https://github.com/gboeing/osmnx-examples/blob/v0.13.0/notebooks/02-routing-speed-time.ipynb
- https://osmnx.readthedocs.io/en/stable/osmnx.html#module-osmnx.speed
   

**add_edge_speeds() function in OSMnx**
- "For highway types in the graph that have no maxspeed value on any edge, it assigns the mean of all maxspeed values in graph".
- "This default mean-imputation can obviously be imprecise, and the user can override it by passing in hwy_speeds and/or fallback arguments that correspond to local speed limit standards. The user can also specify a different aggregation function (such as the median) to impute missing values from the observed values".

Please note, for simplicity, this analysis assumes the vehicle making the deliveries will always be traveling at the maximum speed. 

In [ ]:
# firstly, lets see the distribution of the maxspeeds in the network to determine whether using mean-imputation is viable

# copying the graph edges dataframe
edges_ms_cleaned = edges.copy()

# using the errors='coerce' parameter to convert non-numeric 'maxspeed' values to NaN
# adding the values to the 'edges_ms_cleaned' dataframe
edges_ms_cleaned["maxspeed_cleaned"] = pd.to_numeric(edges_ms_cleaned['maxspeed'], errors='coerce')

In [ ]:
# plotting a histogram of maxspeed
edges_ms_cleaned.hist(column = 'maxspeed_cleaned')

# maxspeed has a relatively normal distribution, therefore mean-imputation is viable method for creating synthetic data.

Now let's complete mean-imputation for the missing data

In [ ]:
# impute speed on all edges missing data
G_Rich = ox.add_edge_speeds(G_Rich)

# calculate travel time (seconds) for all edges
G_Rich = ox.add_edge_travel_times(G_Rich)

In [ ]:
# creating updated geodataframes for nodes and edges in the network with additional attributes
nodes = ox.graph_to_gdfs(G_Rich, nodes=True, edges=False)
edges = ox.graph_to_gdfs(G_Rich, nodes=False, edges=True)

In [ ]:
edges.isna().sum()
# columns 'speed_kph' and 'travel_time' have no null values

# displaying the updated dataframe and checking the results
edges
# mean-imputation was successful

In [ ]:
# displaying mean speed/time values by road type
edges["highway"] = edges["highway"].astype(str)
edges.groupby("highway")[["length", "speed_kph", "travel_time"]].mean().round(1)

Rather than mean-imputation, an alternative apporach may wish to assign maxspeeds manually to each highway type

### RoutePlan() function 

In [ ]:
# creating RoutePlan() function 

def RoutePlan(G, D, L):
    ''' 
    Function to determine the shortest route between a start depot location 
    and a list of delivery locations. 
    
    Where G is a Networkx Graph, D is the Depot
    and L is a list of delivery locations 
    
    Locations are to be store in a list as coordinates [(latitude,longitude)]
    '''
    # combining all coordinates into one list
    locations = D + L
    # convert the lat and long coordinates to nearest node in the graph
    allnodes = [ox.nearest_nodes(G, coord[1], coord[0]) for coord in locations]
    
    # initialise the route with the first node in the list
    route = [allnodes[0]]

    # iterate through the list of nodes and find the shortest path between each consecutive pair
    # and add each iteration of nodes to route
    for i in range(1, len(allnodes)):
        sp = ox.shortest_path(G, allnodes[i-1], allnodes[i], weight='travel_time') # weight is set to 'travel_time'
        route.extend(sp[1:])
    
    return route

# 4. RoutePlan returns the route that will complete the deliveries in the shortest possible time.


In [ ]:
# setting the parameters prior to using the RoutePlan() function
depot_location = [(-37.808303, 145.004701)]
delivery_locations = [(-37.816054, 145.001029),
                      (-37.804051, 145.024595), 
                      (-37.838866, 145.022621), 
                      (-37.827949, 144.990051),
                      (-37.806748, 144.987905)]

In [ ]:
# applying the RoutePlan() function and storing the results in the variable route
route = RoutePlan(G_Rich, depot_location, delivery_locations)

# 5. Plot a map of the street network of the selected city with the route highlighted.

In [ ]:
# plotting the street network with the route highlighted
fig, ax = ox.plot_graph_route(G_Rich, route, route_linewidth=6, node_size=0)

In [ ]:
# changing node colour and size

# generating depot and delivery nodes
locations = depot_location + delivery_locations
allnodes = [ox.nearest_nodes(G, coord[1], coord[0]) for coord in locations]

# create a node colours
colours = ['blue' if i in allnodes[1:len(locations)] else \
              'green' if i in allnodes[0:] else \
              'grey' for i in list(G_Rich.nodes())
              ]

# node size
node_size = [200 if i in allnodes[0:len(locations)] else \
             0 for i in list(G_Rich.nodes())
             ]

In [ ]:
# plotting the street network with the route highlighted
# start node (depot location) is coloured green and the delivery nodes are coloured blue
fig, ax = ox.plot_graph_route(G_Rich, route, orig_dest_size=0, route_linewidth=6, node_color = colours, node_size = node_size)

In [ ]:
# plotting the route on an interactive folium map
ox.folium.plot_route_folium(G_Rich, route, route_map=st_net_m, color="#8b0000", popup_attribute='speed_kph')

Let's contrast shortest paths by length (yellow) vs by travel time (red):

In [ ]:
# creating RoutePlan_length() function 

def RoutePlan_length(G, D, L):
    ''' 
    Same as the RoutePlan() function but uses edge length as the weight
    '''
    
    # combining all lat and lon into one list
    locations = D + L
    # Convert the lat-long coordinates to nearest node in the graph
    allnodes = [ox.nearest_nodes(G, coord[1], coord[0]) for coord in locations]
    
    # Initialize the route with the first node
    route = [allnodes[0]]

    # Iterate through the list of nodes and find the shortest path between each consecutive pair
    for i in range(1, len(allnodes)):
        sp = ox.shortest_path(G, allnodes[i-1], allnodes[i], weight='length')
        route.extend(sp[1:])
    
    return route

In [ ]:
# applying the RoutePlan() and RoutePlan_Length() functions 
route1 = RoutePlan_length(G_Rich, depot_location, delivery_locations)
route2 = RoutePlan(G_Rich, depot_location, delivery_locations)

In [ ]:
# plotting the routes

fig, ax = ox.plot_graph_routes(
    G, routes=[route1, route2], route_colors=["y", "r"], orig_dest_size=0, route_linewidth=6, node_color = colours, node_size = node_size)

- Please note, an orange route indicates the deliveries are following the same path.

An interesting difference between the 2 routes can be seen in the south, where the travel_time weighted route, coloured red, uses the CityLink motor which has a max speed of 80kph. Conversely, the length weighted route, uses the secondary road to the north, Swan Street, which has a maxspeed of 40kph.

### Traveling Salesman Problem (TSP) - additional research

Traveling Salesman Problem (TSP) - 'Given a set of locations and the distance between every pair of locations, the problem is to find the shortest possible route that visits each location exactly once and returns to the original location'.

A down fall of the RoutePlan() function is it relies on the delivery locations being visited in a specific order. What if we want to visit all delivery locations in the fastest time possible, thereby visiting the delivery locations in any order. This is a typical TSP.

Using the tsp() function, G needs to be strongly connected. A directed graph is strongly connected if and only if every vertex in the graph is reachable from every other vertex). Given we are working with a street network, directionality within the network results in not all nodes between reachable. To get around this we must make the graph undirected.

https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.components.is_strongly_connected.html#networkx.algorithms.components.is_strongly_connected


traveling_salesman_problem() function in networkx
https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.approximation.traveling_salesman.traveling_salesman_problem.html


In [ ]:
G_Rich_tsp = G_Rich.copy()

# making the graph undirected
G_Rich_tsp = G_Rich_tsp.to_undirected()

locations = depot_location + delivery_locations 
# convert the lat-lon coordinates to nearest node in the graph
allnodes = [ox.nearest_nodes(G_Rich_tsp, coord[1], coord[0]) for coord in locations]


In [ ]:
route_tsp = nx.approximation.traveling_salesman_problem(G_Rich_tsp, nodes=allnodes, weight='travel_time')

In [ ]:
route_tsp

In [ ]:
# plotting route_tsp
fig, ax = ox.plot_graph_route(G_Rich_tsp, route_tsp, route_color='red', route_linewidth=6, node_size=1, bgcolor='k')

Although this route returns to the origin node (cycle=True is default) and ignores edge directionality, it's still interesting to compare the route_tsp to the previous routes. 